In [ ]:
from pdg_methods import birge_ratio, symmetrize_error
import numpy as np
import matplotlib.pyplot as plt

from pdg_data import load_pdg_data
_, pdg2025_dfs, _, _ = load_pdg_data()

# Checking sensitivity to iterative algorithm initialization

In [ ]:
def dfplot(df):
    plt.errorbar(df.measurement, np.arange(len(df)), xerr=np.array([df.error_negative, df.error_positive]), color='black', fmt='.')

In [ ]:
# demonstration of different convergence depending on init
plt.figure(figsize=(3,2))
values = np.array([0, 1])
error_n = np.array([0.1, 1])
error_p = np.array([1.0, 0.1])
plt.errorbar(values, np.arange(len(values)), xerr=np.array([error_n, error_p]), fmt='.', color='black', label='data')
wm1, wm1_err_n, wm1_err_p = birge_ratio(values, error_n, error_p, init=0.2)
wm2, wm2_err_n, wm2_err_p = birge_ratio(values, error_n, error_p, init=0.8)
plt.errorbar([wm1, wm2], [2,2], xerr=np.array([[wm1_err_n, wm2_err_n],[wm1_err_p,wm2_err_p]]), fmt='.', color='red', label='wm')
plt.text(s='init 0.2', x=wm1, y=2, va='top', ha='center')
plt.text(s='init 0.8', x=wm2, y=2, va='top', ha='center')
plt.yticks([])
plt.legend()
plt.ylim(None,4)
plt.show()

In [ ]:
wm0s = []
wm1s = []
meanerrs = []
for df in pdg2025_dfs:
    value = np.array(df.measurement)
    error_n = np.array(df.error_negative)
    error_p = np.array(df.error_positive)
    meanerrs.append(np.mean(error_n+error_p))

    wm0, _, _ = birge_ratio(value, error_n, error_p, init=np.min(value))
    wm1, _, _ = birge_ratio(value, error_n, error_p, init=np.max(value))
    wm0s.append(wm0)
    wm1s.append(wm1)
wm0s = np.array(wm0s)
wm1s = np.array(wm1s)
meanerrs = np.array(meanerrs)

In [ ]:
print(np.max(np.abs(wm1s-wm0s)/meanerrs))
pdg2025_dfs[np.argmax(np.abs(wm1s-wm0s)/meanerrs)]

In [ ]:
diffs = np.linspace(-2, 1, 200)
errors = np.full(diffs.shape, np.nan)
for i in range(len(diffs)):
    errors[i] = symmetrize_error(np.array([diffs[i]]), np.array([1]), np.array([0.5]))[0]
plt.plot(diffs, errors)
plt.axvline(0, color='red')
plt.ylim(0,1.1)
print(errors[np.argmin(np.abs(diffs-0))])

# Weighting in final error calculation

In [ ]:
plt.figure(figsize=(4,3))
values = np.array([0,1])
error_n = np.array([0.1, 1])
error_p = np.array([1, 0.1])
plt.errorbar(values, np.arange(len(values)), xerr=np.array([error_n, error_p]), fmt='.', color='black', label='data')
wm, wm_err_n, wm_err_p = birge_ratio(values, error_n, error_p, pdg_err=True)
plt.errorbar([wm], [2], xerr=[[wm_err_n],[wm_err_p]], fmt='.', label='PDG weights')
print(wm_err_n, wm_err_p)
wm, wm_err_n, wm_err_p = birge_ratio(values, error_n, error_p, pdg_err=False)
plt.errorbar([wm], [3], xerr=[[wm_err_n],[wm_err_p]], fmt='.', label='alt. weights')
print(wm_err_n, wm_err_p)
plt.legend()
plt.ylim(None,5)

In [ ]:
wms = []
interval0s = []
interval1s = []
width0s = []
width1s = []
for df in pdg2025_dfs:
    value = np.array(df.measurement)
    error_n = np.array(df.error_negative)
    error_p = np.array(df.error_positive)
    wm0, wm0_err_n, wm0_err_p = birge_ratio(value, error_n, error_p, pdg_err=True)
    interval0s.append([wm0-wm0_err_n, wm0+wm0_err_p])
    width0s.append(wm0_err_n + wm0_err_p)
    wm1, wm1_err_n, wm1_err_p = birge_ratio(value, error_n, error_p, pdg_err=False)
    interval1s.append([wm1-wm1_err_n, wm1+wm1_err_p])
    width1s.append(wm1_err_n + wm1_err_p)

    assert wm0 == wm1
    wms.append(wm0)

width0s = np.array(width0s)
width1s = np.array(width1s)

In [ ]:
pdg2025_dfs[np.argmax(width1s/width0s)]

In [ ]:
ratios = width1s/width0s
different = ~np.isclose(ratios,1)
plt.hist(ratios[different], bins=np.linspace(0.8,1.2,100))
# plt.yscale('log')
plt.show()

In [ ]:
idx = np.argsort(ratios)[-21]
print(ratios[idx])
print(pdg2025_dfs[idx])
dfplot(pdg2025_dfs[idx])
plt.axvspan(interval0s[idx][0], interval0s[idx][1], color='grey')
plt.axvspan(interval1s[idx][0], interval1s[idx][1], color='red', alpha=0.2, linewidth=0)
plt.axvline(wms[idx], color='red')

# Birge Ratio Confidence Interval

In [ ]:
from pdg_methods import birge_ratio_conf

br_ints = []
brs = []
for df in pdg2025_dfs:
    _, _, _, br, br_int = birge_ratio(df.measurement, df.error_negative, df.error_positive)
    brs.append(br)
    br_ints.append(br_int)

In [ ]:
idx = 17
dfplot(pdg2025_dfs[idx])
print(brs[idx])
print(br_ints[idx])
print(pdg2025_dfs[idx])